### Build fragments

In [1]:
import pandas as pd
import json
from pathlib import Path

from tg.grammar_ru import CorpusReader
from tg.projects.book_tasks.fragments_builder import FragmentsBuilder
from pathlib import Path

In [ ]:
martin_reader = CorpusReader(Path('../retell/featurized_corpuses/books/eng/Martin.featurized.books.corpus.zip'))

frame = martin_reader.get_frames().first()
# frame[frame["sentence_id"] == 9]

count_extra_sentences = 0
extra_sentences_id = []
for sentence_id in frame["sentence_id"].unique():
    stop_counter = 0
    for word in frame[frame["sentence_id"] == sentence_id]["word"]:
        if word == '.':
            stop_counter += 1
    if stop_counter > 1:
        extra_sentences_id.append((sentence_id, stop_counter - 1))
        count_extra_sentences += 1

print(count_extra_sentences)
print(extra_sentences_id)

print(frame[frame["sentence_id"] == 371])

Creating fragments of Martin books

In [ ]:
corpus = Path('../retell/featurized_corpuses/books/eng/Martin.featurized.books.corpus.zip')
fragments_builder = FragmentsBuilder(corpus, file_name="martin_fragments_1")
fragments_builder.construct_fragments_json()

In [ ]:
with open(Path('./fragments/martin_fragments.json')) as json_file:
    json_data = json.load(json_file)

    for fragment in json_data['fragments']:
        print(fragment['text'])

In [7]:
corpus = Path('./files/corpus.base.zip')
corpus_reader = CorpusReader(corpus)

frame = corpus_reader.get_frames().first()
frame

,word_id,sentence_id,word_index,paragraph_id,word_tail,word,word_type,word_length,file_id,corpus_id
0,0,0,0,0,0,Севастополь,ru,11,ada4ba73-532a-4fa9-9c27-3e1ef506792a,corpus.base.zip
1,1,0,1,0,0,.,punct,1,ada4ba73-532a-4fa9-9c27-3e1ef506792a,corpus.base.zip
2,2,1,0,1,1,День,ru,4,ada4ba73-532a-4fa9-9c27-3e1ef506792a,corpus.base.zip
3,3,1,1,1,1,15,unk,2,ada4ba73-532a-4fa9-9c27-3e1ef506792a,corpus.base.zip
4,4,1,2,1,0,июля,ru,4,ada4ba73-532a-4fa9-9c27-3e1ef506792a,corpus.base.zip
...,...,...,...,...,...,...,...,...,...,...
199,199,26,5,20,1,делового,ru,8,ada4ba73-532a-4fa9-9c27-3e1ef506792a,corpus.base.zip
200,200,26,6,20,0,человека,ru,8,ada4ba73-532a-4fa9-9c27-3e1ef506792a,corpus.base.zip
201,201,26,7,20,0,.,punct,1,ada4ba73-532a-4fa9-9c27-3e1ef506792a,corpus.base.zip
202,202,27,0,21,0,Разошлись,ru,9,ada4ba73-532a-4fa9-9c27-3e1ef506792a,corpus.base.zip


open-chat-api

In [ ]:
import requests

url = "http://127.0.0.1:5000/v1/chat/completions"

headers = {
    "Content-Type": "application/json"
}

history = []
json_file = open(Path('./fragments/martin_fragments.json'))
json_data = json.load(json_file)
json_file.close()

for fragment in json_data['fragments']:
    print(fragment['text'])
    user_message = fragment['text']
    history.append({"role": "user", "content": user_message})
    data = {
        "model": "openchat_3.5.Q5_K_M.gguf",
        "messages": history
    }

    response = requests.post(url, headers=headers, json=data, verify=False)
    assistant_message = response.json()['choices'][0]['message']['content']
    history.append({"role": "assistant", "content": assistant_message})
    print(assistant_message)
    fragment['retell'] = assistant_message

with open(Path('./fragments/martin_fragments.json'), 'w') as json_file:
    json.dump(json_file)